# 📊 Interprétation des Résultats MMM

Ce notebook analyse les résultats du modèle MMM bayésien :
- Diagnostics de convergence
- Distribution des paramètres
- Attribution par canal
- Insights actionnables

**Auteur** : Ivan  
**Projet** : MMM Bayésien - MSMIN5IN43

In [ ]:
# Imports
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

from data.loader import create_sample_data
from models.base_mmm import BayesianMMM
from inference.diagnostics import check_convergence
from visualization.posterior_plots import (
    plot_trace, plot_posterior, plot_forest, plot_energy
)
from visualization.contribution import (
    calculate_channel_contributions,
    plot_contribution_bars,
    plot_contribution_pie,
    plot_contribution_waterfall,
    plot_response_curves
)

# Config
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
%matplotlib inline

np.random.seed(42)

## 1️⃣ Charger les données et entraîner le modèle

In [ ]:
# Générer données
df = create_sample_data(n_periods=80, n_media_channels=3, seed=42)

# Extraire sans normalisation
media_cols = ['media_1_spend', 'media_2_spend', 'media_3_spend']
X_media = df[media_cols].values
y = np.log1p(df['sales'].values)
dates = df['date']

print(f"✓ Données: {len(df)} périodes")
print(f"✓ X_media: {X_media.shape}")
print(f"✓ y: {y.shape}")

In [ ]:
# Entraîner le modèle complet
print("🔨 Entraînement du modèle complet (adstock + saturation)")
print("   Cela prend ~2-3 minutes...\n")

mmm = BayesianMMM(use_adstock=True, use_saturation=True)

# Paramètres de transformation
alpha = np.array([0.5, 0.6, 0.4])
k = X_media.mean(axis=0)
s = np.array([1.0, 1.0, 1.0])

trace = mmm.fit(
    X_media, y,
    alpha=alpha, k=k, s=s,
    draws=1000,
    tune=1000,
    chains=2,
    random_seed=42
)

print("\n✅ Modèle entraîné !")

## 2️⃣ Diagnostics de convergence

In [ ]:
# Vérifier la convergence
report = check_convergence(trace)

print("📊 Rapport de convergence\n")
print(f"Convergence: {'✅ OUI' if report['converged'] else '❌ NON'}")
print(f"R-hat max: {report['r_hat_max']:.4f} (cible < 1.01)")
print(f"ESS bulk min: {report['ess_bulk_min']:.0f} (cible > 400)")
print(f"ESS tail min: {report['ess_tail_min']:.0f} (cible > 400)")
print(f"Divergences: {report['n_divergences']}")

if report['warnings']:
    print("\n⚠️  Warnings:")
    for w in report['warnings']:
        print(f"  - {w}")

In [ ]:
# Trace plots
fig = plot_trace(trace)
plt.show()

In [ ]:
# Energy plot
fig = plot_energy(trace)
plt.show()

## 3️⃣ Distributions a posteriori

In [ ]:
# Résumé statistique
summary = mmm.summary()
print("📈 Résumé des paramètres\n")
print(summary[['mean', 'sd', 'hdi_3%', 'hdi_97%', 'r_hat', 'ess_bulk']].round(3))

In [ ]:
# Distributions posteriors
fig = plot_posterior(trace)
plt.show()

In [ ]:
# Forest plot des coefficients
fig = plot_forest(trace)
plt.show()

## 4️⃣ Attribution par canal

In [ ]:
# Appliquer les transformations pour calculer contributions
X_transformed = mmm.apply_transformations(X_media, alpha, k, s)

# Calculer les contributions
channel_names = ['TV', 'Facebook', 'Google Ads']
contributions = calculate_channel_contributions(trace, X_transformed, channel_names)

print("💰 CONTRIBUTIONS PAR CANAL\n")
print(contributions[['channel', 'total_contribution', 'pct_total', 'coefficient']].round(2))

In [ ]:
# Barplot
fig = plot_contribution_bars(contributions)
plt.show()

In [ ]:
# Pie chart
fig = plot_contribution_pie(contributions)
plt.show()

In [ ]:
# Waterfall chart
intercept = trace.posterior['intercept'].mean().values
fig = plot_contribution_waterfall(contributions, baseline=intercept)
plt.show()

## 5️⃣ Courbes de réponse

In [ ]:
# Courbes de réponse pour chaque canal
for i, name in enumerate(channel_names):
    spend_range = np.linspace(0, X_media[:, i].max() * 1.5, 100)
    fig = plot_response_curves(
        spend_range,
        alpha=alpha[i],
        k=k[i],
        s=s[i],
        channel_name=name
    )
    plt.show()

## 6️⃣ Performance du modèle

In [ ]:
# Prédictions vs réalité
y_pred = mmm.predict(X_media)

# Métriques
mae = np.mean(np.abs(y - y_pred))
rmse = np.sqrt(np.mean((y - y_pred) ** 2))
mape = np.mean(np.abs((y - y_pred) / y)) * 100

print("📊 Performance du modèle\n")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
# Visualisation prédictions vs réalité
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(dates, y, label='Ventes réelles (log)', linewidth=2, alpha=0.7)
ax.plot(dates, y_pred, label='Prédictions', linewidth=2, linestyle='--', alpha=0.7)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Ventes (log)', fontsize=12)
ax.set_title('Prédictions vs Réalité', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(y, y_pred, alpha=0.6, s=50)
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', linewidth=2, label='Perfect fit')

ax.set_xlabel('Ventes réelles (log)', fontsize=12)
ax.set_ylabel('Prédictions (log)', fontsize=12)
ax.set_title(f'Scatter Plot (MAE={mae:.4f})', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7️⃣ Insights actionnables

### 📊 Résumé des résultats

In [ ]:
# Récapitulatif
print("="*60)
print("INSIGHTS CLÉS - MODÈLE MMM")
print("="*60)
print()

# 1. Performance
print("1️⃣ PERFORMANCE DU MODÈLE")
print(f"   MAE: {mae:.4f} | RMSE: {rmse:.4f} | MAPE: {mape:.1f}%")
print(f"   Convergence: {'✅' if report['converged'] else '❌'}")
print()

# 2. Attribution
print("2️⃣ ATTRIBUTION PAR CANAL")
for _, row in contributions.iterrows():
    print(f"   {row['channel']:15s}: {row['pct_total']:5.1f}% des ventes")
print()

# 3. Paramètres de transformation
print("3️⃣ PARAMÈTRES DES TRANSFORMATIONS")
params = mmm.get_transformation_params()
for i, name in enumerate(channel_names):
    a = params['adstock']['alpha'][i]
    persistence = int(1/(1-a)) if a < 1 else np.inf
    print(f"   {name}:")
    print(f"     - Adstock (α): {a:.2f} → effet persiste ~{persistence} périodes")
    print(f"     - Half-saturation (k): {params['saturation']['k'][i]:.0f}€")
print()

# 4. Recommandations
print("4️⃣ RECOMMANDATIONS")
best_channel = contributions.iloc[0]
worst_channel = contributions.iloc[-1]
print(f"   ✅ Canal le plus performant: {best_channel['channel']} ({best_channel['pct_total']:.1f}%)")
print(f"   ⚠️  Canal le moins performant: {worst_channel['channel']} ({worst_channel['pct_total']:.1f}%)")
print()
print("   💡 Actions suggérées:")
print(f"      1. Augmenter budget {best_channel['channel']} (ROI élevé)")
print(f"      2. Optimiser ou réduire {worst_channel['channel']}")
print(f"      3. Tester nouvelles stratégies sur canaux intermédiaires")
print()

print("="*60)

## 8️⃣ Conclusions

### ✅ Ce qu'on a appris

1. **Attribution des ventes** : Chaque canal contribue différemment aux ventes
2. **Effet d'adstock** : La publicité a un effet persistant dans le temps
3. **Saturation** : Les rendements sont décroissants au-delà d'un certain seuil
4. **Convergence** : Le modèle bayésien a bien convergé

### 🚀 Prochaines étapes

1. **Optimisation budgétaire** : Trouver l'allocation optimale
2. **Scénarios what-if** : Simuler différentes stratégies
3. **Validation croisée** : Tester sur données out-of-sample
4. **Déploiement** : Mettre en production

### 📚 Ressources

- [PyMC Documentation](https://docs.pymc.io/)
- [Bayesian Marketing Mix Modeling](https://www.pymc-marketing.io/)
- [Google LightweightMMM](https://github.com/google/lightweight_mmm)